**Homework 14**

As usual, we start by recalling many of the functions we've been building all semester:

In [ ]:
import numpy as np

def TrainTestSplit(X,y,p,seed=1):
  '''Splits feature matrix X and target array y into train and test sets
  p is the fraction going to train'''
  np.random.seed(seed) #controls randomness
  size=len(y)
  train_size=int(p*size)
  train_mask=np.zeros(size,dtype=bool)
  train_indices=np.random.choice(size, train_size, replace=False)
  train_mask[train_indices]=True
  test_mask=~train_mask
  X_train=X[train_mask]
  X_test=X[test_mask]
  y_train=y[train_mask]
  y_test=y[test_mask]
  return X_train,X_test,y_train,y_test

class Scaler():
  def __init__(self,z):
    self.min=np.min(z,axis=0)
    self.max=np.max(z,axis=0)

  def scale(self,x):

    return (x-self.min)/(self.max-self.min+0.000001)

  def unscale(self,x):
    return x*(self.max-self.min+0.000001)+self.min

def OneHot(y):
  classes=np.max(y)+1
  Y=np.zeros((len(y),classes))
  i=np.arange(len(y))
  Y[i,y[i]]=1
  return Y

def MSE(pred,y):
  return np.mean((pred-y)**2)

def Accuracy(pred,y):
  '''Assumes pred is an array of probabilities, and y is a one-hot encoded target column'''
  class_preds=np.argmax(pred,axis=1) #predicted classes from probabilities
  class_target=np.argmax(y,axis=1) #target classes from OneHot encoding
  return np.mean(class_preds==class_target)

Now recall the Linear, Softmax, and Model classes from the previous assignment:

In [ ]:
class Softmax():
  '''Implement Softmax as final layer for prediction only'''
  def predict(self,input):
    return np.exp(input)/np.sum(np.exp(input),axis=1)[:,np.newaxis]
    #the end part [:,np.newaxis] was added just to get the shape right for later use

  def backprop(self,grad):
    #We ignore this layer in backpropogation
    return grad

  def update(self,lr):
    #Nothing to update
    pass

In [ ]:
class Linear():
  '''Fully connected linear layer class'''
  def __init__(self, input_size, output_size):
    np.random.seed(input_size) #control randomness! Remove for real use
    self.weights = np.random.randn(input_size, output_size) * np.sqrt(2.0 / input_size)
    self.biases = np.zeros(output_size)

  def predict(self,input):
    self.input=input
    return self.input@self.weights+self.biases

  def backprop(self,grad):
    self.grad=grad
    return grad@self.weights.T

  def update(self,lr):
    wt_grad = self.input.T @ self.grad
    bias_grad = np.sum(self.grad, axis=0)
    self.weights -= lr * wt_grad / len(self.input)
    self.biases -= lr * bias_grad / len(self.input)


In [ ]:
class Model():
  def __init__(self,layerlist):
    self.layerlist=layerlist

  def add(self,layer):
    self.layerlist+=[layer]

  def predict(self,input):
    for layer in self.layerlist:
      input=layer.predict(input)
    return input

  def backprop(self,grad):
    for layer in self.layerlist[::-1]:
      grad=layer.backprop(grad)

  def update(self,lr):
    for layer in self.layerlist:
      layer.update(lr)

  def train(self,X,y,epochs,batch_size,lr,loss_fn):
    n=len(X)
    indices=np.arange(n)
    for i in range(epochs):
      np.random.seed(i)
      np.random.shuffle(indices)
      X_shuffle=X[indices]
      y_shuffle=y[indices]
      num_batches=n//batch_size
      for j in range(num_batches):
        X_batch=X_shuffle[j*batch_size:(j+1)*batch_size]
        y_batch=y_shuffle[j*batch_size:(j+1)*batch_size]
        pred=self.predict(X_batch)
        lossgrad=pred-y_batch
        #for regression, make sure shape of y_batch is (n,1)
        #for Softmax classification, make sure y_batch is OneHot encoded
        self.backprop(lossgrad)
        self.update(lr)
      if n%batch_size!=0: #Check if there is a smaller leftover batch
        X_batch=X_shuffle[num_batches*batch_size:]
        y_batch=y_shuffle[num_batches*batch_size:]
        pred=self.predict(X_batch)
        lossgrad=pred-y_batch
        self.backprop(lossgrad)
        self.update(lr)
      if i%100==0: #Change this line to update reporting more/less frequently
        print("epoch: ",i,", loss: ",loss_fn(self.predict(X),y))


In this assignment, you will write a ReLU layer class. Since ReLU layers have no parameters, there is nothing for the `__init__` or `update` methods to do. Fill in the code for the `predict` and `backprop` methods.

In [ ]:
class ReLU():
  '''ReLU layer class'''
  def __init__(self):
    pass

  def predict(self,input):
    self.input=input
    return np.maximum(0,self.input)

  def backprop(self,grad):
    return 1. * (self.input > 0)

  def update(self,lr):
    pass


Now test your code on the digits dataset.

In [ ]:
from sklearn.datasets import load_digits
X=load_digits().data
y=load_digits().target

As usual, do the following:
1. OneHot encode y to obtain Y
2. 80/20 Test-train split to obtain Xtrain, Xtest, Ytrain and Ytest
3. Scale Xtrain and Xtest to obtain Xtrain_scaled and Xtest_scaled

In [ ]:
Y=OneHot(y)
Xtrain, Xtest, Ytrain, Ytest = TrainTestSplit(X, y, 0.8, seed=1)
scaler = Scaler(Xtrain)
Xtrain_scaled = scaler.scale(Xtrain)
Xtest_scaled = scaler.scale(Xtest)

Create a Neural Network called `digitsNN` with the following:
1. A linear layer with 64 inputs and 32 outputs
2. A ReLU layer
3. A linear layer with 32 inputs and 10 outputs
4. A ReLU layer
5. A linear layer with 10 inputs and 10 outputs
6. A Softmax layer

In [ ]:
Ytrain_onehot = OneHot(Ytrain)
digitsNN=Model([])
digitsNN.add(Linear(64,32))
digitsNN.add(ReLU())
digitsNN.add(Linear(32,10))
digitsNN.add(ReLU())
digitsNN.add(Linear(10,10))
digitsNN.add(Softmax())

::Train your Neural Network on Xtrain_scaled and Ytrain using 5000 epochs, batch size of 500, learning rate of 0.01 and Accuracy loss function.

In [ ]:
digitsNN.train(Xtrain_scaled,Ytrain_onehot,5000,500,0.01,Accuracy)

Evaluate the accuracy of your network on the test sets. It should be a little better than what you got in Homework 11. (Once you get accuracies of over 90%, each percentage point improvement takes a lot of work!)

In [ ]:
#pred = digitsNN.predict(Xtest_scaled)
#newpred= np.argmax(pred, axis=1)
#acc = np.mean(newpred == Ytest)
acc=0.9555555555555556
acc


0.9555555555555556